In [1]:
import pandas as pd
import datetime

In [2]:
DATA_KAGGLE_RAW_PATH = "../raw_data/data_thai_kaggle.xlsx"
DATA_THAI_GOV_2023_DEC = "../raw_data/data_thai_gov_2023_dec.xlsx"
OUTPUT_DATA_FILE = "../data/collected_data.csv"

In [3]:
# Load thai kaggle data 
original_data = pd.read_excel(DATA_KAGGLE_RAW_PATH)

In [4]:
original_data

,date,province_thai,province_eng,region_thai,region_eng,variable,value
0,2019-01-01,กรุงเทพมหานคร,Bangkok,ภาคกลาง,central,ratio_tourist_stay,93.37
1,2019-01-01,ลพบุรี,Lopburi,ภาคกลาง,central,ratio_tourist_stay,61.32
2,2019-01-01,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,ภาคกลาง,central,ratio_tourist_stay,73.37
3,2019-01-01,สระบุรี,Saraburi,ภาคกลาง,central,ratio_tourist_stay,67.33
4,2019-01-01,ชัยนาท,Chainat,ภาคกลาง,central,ratio_tourist_stay,79.31
...,...,...,...,...,...,...,...
35723,2023-10-01 00:00:00,ร้อยเอ็ด,Roi Et,ภาคตะวันออกเฉียงเหนือ,east_northeast,revenue_foreign,1920000.00
35724,2023-10-01 00:00:00,ศรีสะเกษ,Sisaket,ภาคใต้,south,revenue_foreign,700000.00
35725,2023-10-01 00:00:00,สุรินทร์,Surin,ภาคตะวันออกเฉียงเหนือ,east_northeast,revenue_foreign,4000000.00
35726,2023-10-01 00:00:00,อำนาจเจริญ,Amnat Charoen,ภาคตะวันออกเฉียงเหนือ,east_northeast,revenue_foreign,720000.00


In [5]:
original_data["date"] = pd.to_datetime(original_data["date"])

In [6]:
map_sheet_name_date_time = {"ม.ค.2566(P)": datetime.datetime(2023, 1, 1),
                            "ก.พ.2566(P)": datetime.datetime(2023, 2, 1),
                            "มี.ค.2566(P)": datetime.datetime(2023, 3, 1),
                            "เม.ย.2566(P)": datetime.datetime(2023, 4, 1),
                            "พ.ค.2566(P)": datetime.datetime(2023, 5, 1),
                            "มิ.ย.2566(P)": datetime.datetime(2023, 6, 1),
                            "ก.ค.2566(P)": datetime.datetime(2023, 7, 1),
                            "ส.ค.2566(P)": datetime.datetime(2023, 8, 1),
                            "ก.ย.2566(P2)": datetime.datetime(2023, 9, 1),
                            "ต.ค.2566(P2)": datetime.datetime(2023, 10, 1)
                            }

map_variable_to_column = {"ratio_tourist_stay": 2, "no_tourist_stay": 5, "no_tourist_all":  8, 
                            "no_tourist_thai": 11, "no_tourist_foreign": 14, "revenue_all": 17,
                            "revenue_thai": 20, "revenue_foreign": 23 }
                            
metrics = ["revenue_all", "revenue_thai", "revenue_foreign"]
metric_value = 1e6

for map_sheet in map_sheet_name_date_time:
    data_to_processed = pd.read_excel(DATA_THAI_GOV_2023_DEC, sheet_name=map_sheet, header=3) #2023_1
    for variable in map_variable_to_column:
        column  = map_variable_to_column[variable]
        time_factor = 1
        if variable in metrics:
            time_factor = 1e6
            
        central_data =  data_to_processed.iloc[:18, column] 
        east_data = data_to_processed.iloc[19:27, column] 
        south_data = data_to_processed.iloc[28:42, column] 
        north_data = data_to_processed.iloc[43:60, column] 
        east_northeast_data = data_to_processed.iloc[61:81, column] 
        col  = pd.concat([central_data, east_data, south_data,north_data,east_northeast_data])
        
        index_now = original_data.index[(original_data["date"] == map_sheet_name_date_time[map_sheet]) & (original_data["variable"] == variable)].tolist()
        original_data.loc[index_now, "value"] = [dump_value * time_factor for dump_value in col.to_list()]    


In [7]:
original_data

,date,province_thai,province_eng,region_thai,region_eng,variable,value
0,2019-01-01,กรุงเทพมหานคร,Bangkok,ภาคกลาง,central,ratio_tourist_stay,93.37
1,2019-01-01,ลพบุรี,Lopburi,ภาคกลาง,central,ratio_tourist_stay,61.32
2,2019-01-01,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,ภาคกลาง,central,ratio_tourist_stay,73.37
3,2019-01-01,สระบุรี,Saraburi,ภาคกลาง,central,ratio_tourist_stay,67.33
4,2019-01-01,ชัยนาท,Chainat,ภาคกลาง,central,ratio_tourist_stay,79.31
...,...,...,...,...,...,...,...
35723,2023-10-01,ร้อยเอ็ด,Roi Et,ภาคตะวันออกเฉียงเหนือ,east_northeast,revenue_foreign,2410000.00
35724,2023-10-01,ศรีสะเกษ,Sisaket,ภาคใต้,south,revenue_foreign,860000.00
35725,2023-10-01,สุรินทร์,Surin,ภาคตะวันออกเฉียงเหนือ,east_northeast,revenue_foreign,4440000.00
35726,2023-10-01,อำนาจเจริญ,Amnat Charoen,ภาคตะวันออกเฉียงเหนือ,east_northeast,revenue_foreign,740000.00


In [8]:
original_data.to_csv(OUTPUT_DATA_FILE, index=False)